**Meta introduced and launched the Meta Llama 3** series of large language models (LLMs), comprising pretrained and instruction-tuned generative text models in sizes of 8B and 70B. The instruction-tuned Llama 3 models are specifically tailored for dialogue applications, showcasing superior performance compared to numerous open-source chat models across prevalent industry benchmarks.

In [1]:
# Install Pytorch & other libraries
# Install Hugging Face libraries
!pip install  --upgrade  --quiet\
  "transformers==4.38.2" \
  "datasets==2.16.1" \
  "accelerate==0.26.1" \
  "evaluate==0.4.1" \
  "bitsandbytes==0.42.0" \
  "trl==0.7.11" \
  "peft==0.8.2" \
  "langchain" \
  "sentence-transformers" \
  "faiss-cpu"
!pip install unstructured pdfminer pdfminer.six faiss-cpu "torch==2.1.2" tensorboard --quiet
!pip install -U langchain-community==0.2.4  --quiet


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


# ***### Kernal restart recommended after installing library ***

In [2]:
##importing library
from transformers import (AutoTokenizer,AutoModelForCausalLM,
                          TextStreamer,pipeline,BitsAndBytesConfig)
import transformers
import torch
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [3]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)


/opt/app-root/lib64/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [5]:
####### prompt template




In [6]:
def get_inference(prompt="",max_new_tokens=256,temperature=0.7,top_p=0.9):
    sys_pmt = "Your task is to generate a brief summary of a given text, providing the main points and key details while maintaining coherence and clarity. The summary should be concise and capture the essence of the original text accurately."
    messages = [
    {"role": "system", "content": sys_pmt},
    {"role": "user", "content": prompt},
    ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)
    outputs = pipeline(
        prompt,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )
    return outputs[0]["generated_text"][len(prompt):]

In [7]:
result = get_inference("Write a short summary about photosynthesis.")
print(result.strip())

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
/opt/app-root/lib64/python3.9/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Here is a brief summary about photosynthesis:

Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process occurs in specialized organelles called chloroplasts, which contain the pigment chlorophyll. Chlorophyll absorbs light energy, which is then used to power the conversion of carbon dioxide and water into glucose and oxygen. Oxygen is released as a byproduct, while glucose is used by the plant as energy and building blocks for growth and development. Photosynthesis is essential for life on Earth, as it provides the energy and organic compounds necessary to support the food chain and ecosystem.
